In [ ]:
!pip install scispacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.7 MB/s 
     |████████████████████████████████| 13.5 MB 13.8 MB/s 
     |████████████████████████████████| 71 kB 5.8 MB/s 
     |████████████████████████████████| 6.0 MB 59.8 MB/s 
     |████████████████████████████████| 188 kB 59.1 MB/s 
     |████████████████████████████████| 660 kB 54.4 MB/s 
     |████████████████████████████████| 10.1 MB 35.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.1
    Uninstalling pydantic-1.9.1:
      Successfully uninstalled pydantic-1.9.1
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc

In [ ]:
import spacy

In [ ]:
## Load the Libraries 
import re
import spacy
import random
import scispacy
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from spacy import displacy
from google.colab import drive
import matplotlib.pyplot as plt
from spacy.training.example import Example
from spacy.util import minibatch, compounding


# Load the SCI-SPACY model
nlp = spacy.load('drug_salt_model')

# Getting the ner component
ner=nlp.get_pipe('ner')

In [ ]:
### Load the data
df = pd.read_csv("drug_intro.csv")

In [ ]:
df.head()

,drug_brandname,drug_salt_form,introduction
0,Ascoril D Plus Syrup Sugar Free,Phenylephrine 5mg + Chlorpheniramine Maleate 2...,Ascoril D Plus Syrup Sugar Free is a combinati...
1,Allegra 180mg Tablet,Fexofenadine 180mg,Allegra 180mg Tablet is an anti-allergy medici...
2,Azithral 500 Tablet,Azithromycin 500mg,Azithral 500 Tablet is an antibiotic used to t...
3,Augmentin 625 Duo Tablet,Amoxycillin 500mg + Clavulanic Acid 125mg,Augmentin 625 Duo Tablet is a penicillin-type ...
4,Amoxyclav 625 Tablet,Amoxycillin 500mg + Clavulanic Acid 125mg,Amoxyclav 625 Tablet is a penicillin-type of a...


In [ ]:
df.shape

(207685, 3)

In [ ]:
df['introduction'] = df['introduction'].str.replace(',' , ' ')
df['introduction'] = df['introduction'].str.replace('/n' , ' ')
df['introduction'] = df['introduction'].str.replace(' ' , ' ')

df['drug_salt_form'] = df['drug_salt_form'].str.replace(',' , ' ')
df['drug_salt_form'] = df['drug_salt_form'].str.replace('/n' , ' ')
df['drug_salt_form'] = df['drug_salt_form'].str.replace('.' , ' ')
#df['drug_salt_form'] = df['drug_salt_form'].str.replace('(\d+mg)' , ' ')

df['drug_salt_form'] = df['drug_salt_form'].str.replace('\d+' , ' ')
df['drug_salt_form'] = df['drug_salt_form'].str.replace('mg' , ' ')
# df['drug_salt_form'] = df['drug_salt_form'].str.replace('( )' , ' ')
df.drug_salt_form = df.drug_salt_form.str.replace(' ', '')
df['drug_salt_form'] = df['drug_salt_form'].str.replace('+' ,  ' ')





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [ ]:
df
df.head()

,drug_brandname,drug_salt_form,introduction
0,Ascoril D Plus Syrup Sugar Free,Phenylephrine ChlorpheniramineMaleate Dextrome...,Ascoril D Plus Syrup Sugar Free is a combinati...
1,Allegra 180mg Tablet,Fexofenadine,Allegra 180mg Tablet is an anti-allergy medici...
2,Azithral 500 Tablet,Azithromycin,Azithral 500 Tablet is an antibiotic used to t...
3,Augmentin 625 Duo Tablet,Amoxycillin ClavulanicAcid,Augmentin 625 Duo Tablet is a penicillin-type ...
4,Amoxyclav 625 Tablet,Amoxycillin ClavulanicAcid,Amoxyclav 625 Tablet is a penicillin-type of a...


In [ ]:
def replace( introduction , drug_salt_form , drug_brandname):
#introduction = input()
    intro = []
    introduction = introduction.replace(drug_brandname , drug_salt_form)
    intro.append(introduction)
    return intro

In [ ]:
word = []
index_removed = []
for idx , val in tqdm(df.iterrows()):
    try :
        word.append(replace(val['introduction'] , val['drug_salt_form'] , val['drug_brandname']))
    except:
        print(idx)
        index_removed.append(idx)

4454it [00:00, 14457.22it/s]

1763


7569it [00:00, 15134.59it/s]

5101


10592it [00:00, 14733.55it/s]

...

190773it [00:12, 15567.58it/s]

188349


203121it [00:13, 14297.45it/s]

201154
203912


207685it [00:14, 14526.20it/s]


In [ ]:
replace(val['introduction'] , val['drug_salt_form'] , val['drug_brandname'])

['NandroloneDecanoate is a medicine used in the treatment of osteoporosis in post-menopausal women. It helps strengthen the bones which become thin and brittle due to osteoporosis. NandroloneDecanoate is generally given by a healthcare professional. You should not self-administer this medicine at home. Your doctor will decide the dose and how often you should take them. You should take it regularly to get the most benefit from it. Do not stop using the medicine even if you feel better unless the doctor tells you so. Using this medicine may cause few common side effects such as edema swelling of legs  ankle  and feet  fluid retention  nausea  acne  and breast enlargement. Let your doctor know if any of these side effects bother you. Before using this medicine  inform your doctor if you have any liver or kidney problems. You should also tell your doctor all the other medicines you are taking. Pregnant and breastfeeding mothers should consult with their doctors before using the medicine.'

In [ ]:
word

[['Phenylephrine ChlorpheniramineMaleate DextromethorphanHydrobromide is a combination medicine used in the treatment of dry cough. It relieves allergic symptoms such as sneezing  runny nose  watery eyes  and throat irritation. It also provides relief from congestion or stuffiness in the nose. It is taken with or without food in a dose and duration as advised by the doctor. The dose you are given will depend on your condition and how you respond to the medicine. You should keep taking this medicine for as long as your doctor recommends. If you stop treatment too early your symptoms may come back and your condition may worsen. Let your healthcare team know about all other medications you are taking as some may affect  or be affected by this medicine. The most common side effects are nausea  vomiting  loss of appetite  and headache. Most of these are temporary and usually resolve with time. Contact your doctor straight away if you are at all concerned about any of these side effects. Thi

In [ ]:
print(df.shape)
df.drop(index=index_removed, inplace=True)
print(df.shape)

(207685, 3)
(207635, 3)


In [ ]:
df['intro'] = word
df.head()

,drug_brandname,drug_salt_form,introduction,intro
0,Ascoril D Plus Syrup Sugar Free,Phenylephrine ChlorpheniramineMaleate Dextrome...,Ascoril D Plus Syrup Sugar Free is a combinati...,[Phenylephrine ChlorpheniramineMaleate Dextrom...
1,Allegra 180mg Tablet,Fexofenadine,Allegra 180mg Tablet is an anti-allergy medici...,[Fexofenadine is an anti-allergy medicine used...
2,Azithral 500 Tablet,Azithromycin,Azithral 500 Tablet is an antibiotic used to t...,[Azithromycin is an antibiotic used to treat v...
3,Augmentin 625 Duo Tablet,Amoxycillin ClavulanicAcid,Augmentin 625 Duo Tablet is a penicillin-type ...,[Amoxycillin ClavulanicAcid is a penicillin-ty...
4,Amoxyclav 625 Tablet,Amoxycillin ClavulanicAcid,Amoxyclav 625 Tablet is a penicillin-type of a...,[Amoxycillin ClavulanicAcid is a penicillin-ty...


In [ ]:
df

,drug_brandname,drug_salt_form,introduction,intro
0,Ascoril D Plus Syrup Sugar Free,Phenylephrine ChlorpheniramineMaleate Dextrome...,Ascoril D Plus Syrup Sugar Free is a combinati...,[Phenylephrine ChlorpheniramineMaleate Dextrom...
1,Allegra 180mg Tablet,Fexofenadine,Allegra 180mg Tablet is an anti-allergy medici...,[Fexofenadine is an anti-allergy medicine used...
2,Azithral 500 Tablet,Azithromycin,Azithral 500 Tablet is an antibiotic used to t...,[Azithromycin is an antibiotic used to treat v...
3,Augmentin 625 Duo Tablet,Amoxycillin ClavulanicAcid,Augmentin 625 Duo Tablet is a penicillin-type ...,[Amoxycillin ClavulanicAcid is a penicillin-ty...
4,Amoxyclav 625 Tablet,Amoxycillin ClavulanicAcid,Amoxyclav 625 Tablet is a penicillin-type of a...,[Amoxycillin ClavulanicAcid is a penicillin-ty...
...,...,...,...,...
207680,Zinagon 5mg Tablet,Levocetirizine,Zinagon 5mg Tablet belongs to a group of medic...,[Levocetirizine belongs to a group of medicine...
207681,Zinagon M 5mg/10mg Tablet,Levocetirizine Montelukast,Zinagon M 5mg/10mg Tablet is a combination med...,[Levocetirizine Montelukast is a combination m...
207682,Zironi H Tablet,Amitriptyline Chlordiazepoxide,Zironi H Tablet is a combination of two medici...,[Amitriptyline Chlordiazepoxide is a combinati...
207683,Zoesunate 60mg Injection,Artesunate,Zoesunate 60mg Injection is an antiparasitic m...,[Artesunate is an antiparasitic medication. It...


In [ ]:
df['introduc'] = [','.join(map(str, l)) for l in df['intro']]
df

,drug_brandname,drug_salt_form,introduction,intro,introduc
0,Ascoril D Plus Syrup Sugar Free,Phenylephrine ChlorpheniramineMaleate Dextrome...,Ascoril D Plus Syrup Sugar Free is a combinati...,[Phenylephrine ChlorpheniramineMaleate Dextrom...,Phenylephrine ChlorpheniramineMaleate Dextrome...
1,Allegra 180mg Tablet,Fexofenadine,Allegra 180mg Tablet is an anti-allergy medici...,[Fexofenadine is an anti-allergy medicine used...,Fexofenadine is an anti-allergy medicine used ...
2,Azithral 500 Tablet,Azithromycin,Azithral 500 Tablet is an antibiotic used to t...,[Azithromycin is an antibiotic used to treat v...,Azithromycin is an antibiotic used to treat va...
3,Augmentin 625 Duo Tablet,Amoxycillin ClavulanicAcid,Augmentin 625 Duo Tablet is a penicillin-type ...,[Amoxycillin ClavulanicAcid is a penicillin-ty...,Amoxycillin ClavulanicAcid is a penicillin-typ...
4,Amoxyclav 625 Tablet,Amoxycillin ClavulanicAcid,Amoxyclav 625 Tablet is a penicillin-type of a...,[Amoxycillin ClavulanicAcid is a penicillin-ty...,Amoxycillin ClavulanicAcid is a penicillin-typ...
...,...,...,...,...,...
207680,Zinagon 5mg Tablet,Levocetirizine,Zinagon 5mg Tablet belongs to a group of medic...,[Levocetirizine belongs to a group of medicine...,Levocetirizine belongs to a group of medicines...
207681,Zinagon M 5mg/10mg Tablet,Levocetirizine Montelukast,Zinagon M 5mg/10mg Tablet is a combination med...,[Levocetirizine Montelukast is a combination m...,Levocetirizine Montelukast is a combination me...
207682,Zironi H Tablet,Amitriptyline Chlordiazepoxide,Zironi H Tablet is a combination of two medici...,[Amitriptyline Chlordiazepoxide is a combinati...,Amitriptyline Chlordiazepoxide is a combinatio...
207683,Zoesunate 60mg Injection,Artesunate,Zoesunate 60mg Injection is an antiparasitic m...,[Artesunate is an antiparasitic medication. It...,Artesunate is an antiparasitic medication. It ...


In [ ]:
df.drop('intro' , axis=1)

,drug_brandname,drug_salt_form,introduction,introduc
0,Ascoril D Plus Syrup Sugar Free,Phenylephrine ChlorpheniramineMaleate Dextrome...,Ascoril D Plus Syrup Sugar Free is a combinati...,Phenylephrine ChlorpheniramineMaleate Dextrome...
1,Allegra 180mg Tablet,Fexofenadine,Allegra 180mg Tablet is an anti-allergy medici...,Fexofenadine is an anti-allergy medicine used ...
2,Azithral 500 Tablet,Azithromycin,Azithral 500 Tablet is an antibiotic used to t...,Azithromycin is an antibiotic used to treat va...
3,Augmentin 625 Duo Tablet,Amoxycillin ClavulanicAcid,Augmentin 625 Duo Tablet is a penicillin-type ...,Amoxycillin ClavulanicAcid is a penicillin-typ...
4,Amoxyclav 625 Tablet,Amoxycillin ClavulanicAcid,Amoxyclav 625 Tablet is a penicillin-type of a...,Amoxycillin ClavulanicAcid is a penicillin-typ...
...,...,...,...,...
207680,Zinagon 5mg Tablet,Levocetirizine,Zinagon 5mg Tablet belongs to a group of medic...,Levocetirizine belongs to a group of medicines...
207681,Zinagon M 5mg/10mg Tablet,Levocetirizine Montelukast,Zinagon M 5mg/10mg Tablet is a combination med...,Levocetirizine Montelukast is a combination me...
207682,Zironi H Tablet,Amitriptyline Chlordiazepoxide,Zironi H Tablet is a combination of two medici...,Amitriptyline Chlordiazepoxide is a combinatio...
207683,Zoesunate 60mg Injection,Artesunate,Zoesunate 60mg Injection is an antiparasitic m...,Artesunate is an antiparasitic medication. It ...


In [ ]:
df['introduction'].dtype

dtype('O')

In [ ]:
# ### Function to get the start and end point of the drug name
# def get_name (pattern , string):
#     """
#     Function to get the start and end point of the drug name
#     """
#     # Loop across the sentence
#     for i in re.finditer(pattern, string):
#         entity = (i.span())
#         return (entity)

### Function to get the start and end point of the drug name
def get_name(pattern , string):
    """
    Function to get the start and end point of the drug name
    """
    annotate_list = []
    # Loop across the sentence
    for i in re.finditer(pattern, string):
        annotate_list.append((i.span()[0], i.span()[1], "Drug_Salt_Form"))
    return annotate_list

In [ ]:
# Creating a list of vocab
word_range = []
index_to_remove = []

# Loop across all the sentences
for idx , val in tqdm(df.iterrows()):
   try:
      word_range.append((get_name(val["drug_salt_form"] , val["introduc"] )))
   except:
     print(idx)
     print(val["drug_salt_form"] , val["introduc"])
     index_to_remove.append(idx)

207635it [00:19, 10754.09it/s]


In [ ]:
get_name(val["drug_salt_form"] , val["introduc"] )
word_range

[[(0, 66, 'Drug_Salt_Form')],
 [(0, 12, 'Drug_Salt_Form')],
 [(0, 12, 'Drug_Salt_Form')],
 [(0, 26, 'Drug_Salt_Form')],
 [(0, 26, 'Drug_Salt_Form')],
 [(0, 44, 'Drug_Salt_Form')],
 [(0, 10, 'Drug_Salt_Form')],
 [(0, 11, 'Drug_Salt_Form')],
 [(0, 13, 'Drug_Salt_Form')],
 [(0, 11, 'Drug_Salt_Form')],
 [(0, 75, 'Drug_Salt_Form')],
 [(0, 12, 'Drug_Salt_Form')],
 [(0, 12, 'Drug_Salt_Form')],
 [(0, 11, 'Drug_Salt_Form')],
 [(0, 10, 'Drug_Salt_Form')],
 [(0, 24, 'Drug_Salt_Form')],
 [(0, 25, 'Drug_Salt_Form')],
 [(0, 49, 'Drug_Salt_Form')],
 [(0, 26, 'Drug_Salt_Form'), (858, 884, 'Drug_Salt_Form')],
 [(0, 19, 'Drug_Salt_Form')],
 [(0, 75, 'Drug_Salt_Form')],
 [(0, 14, 'Drug_Salt_Form')],
 [(0, 13, 'Drug_Salt_Form')],
 [(0, 15, 'Drug_Salt_Form')],
 [(0, 12, 'Drug_Salt_Form')],
 [(0, 9, 'Drug_Salt_Form')],
 [(0, 10, 'Drug_Salt_Form')],
 [(0, 12, 'Drug_Salt_Form')],
 [(0, 19, 'Drug_Salt_Form')],
 [(0, 12, 'Drug_Salt_Form')],
 [(0, 44, 'Drug_Salt_Form')],
 [(0, 28, 'Drug_Salt_Form')],
 [(0, 12, '

In [ ]:
print(df.shape)
df.drop(index=index_to_remove, inplace=True)
print(df.shape)

(207635, 5)
(207635, 5)


In [ ]:
# Add the data to the df
df["Range"] = word_range

df.head()

,drug_brandname,drug_salt_form,introduction,intro,introduc,Range
0,Ascoril D Plus Syrup Sugar Free,Phenylephrine ChlorpheniramineMaleate Dextrome...,Ascoril D Plus Syrup Sugar Free is a combinati...,[Phenylephrine ChlorpheniramineMaleate Dextrom...,Phenylephrine ChlorpheniramineMaleate Dextrome...,"[(0, 66, Drug_Salt_Form)]"
1,Allegra 180mg Tablet,Fexofenadine,Allegra 180mg Tablet is an anti-allergy medici...,[Fexofenadine is an anti-allergy medicine used...,Fexofenadine is an anti-allergy medicine used ...,"[(0, 12, Drug_Salt_Form)]"
2,Azithral 500 Tablet,Azithromycin,Azithral 500 Tablet is an antibiotic used to t...,[Azithromycin is an antibiotic used to treat v...,Azithromycin is an antibiotic used to treat va...,"[(0, 12, Drug_Salt_Form)]"
3,Augmentin 625 Duo Tablet,Amoxycillin ClavulanicAcid,Augmentin 625 Duo Tablet is a penicillin-type ...,[Amoxycillin ClavulanicAcid is a penicillin-ty...,Amoxycillin ClavulanicAcid is a penicillin-typ...,"[(0, 26, Drug_Salt_Form)]"
4,Amoxyclav 625 Tablet,Amoxycillin ClavulanicAcid,Amoxyclav 625 Tablet is a penicillin-type of a...,[Amoxycillin ClavulanicAcid is a penicillin-ty...,Amoxycillin ClavulanicAcid is a penicillin-typ...,"[(0, 26, Drug_Salt_Form)]"


In [ ]:
### Prepare the training data
TRAIN_DATA = []
LABEL = 'Drug_Salt_Form'
for idx , val in tqdm(df.iterrows()):
    TRAIN_DATA.append(
        (val['introduc'] , {'entities':val["Range"]})
    )

# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

207635it [00:13, 15231.92it/s]


In [ ]:
TRAIN_DATA

[('Phenylephrine ChlorpheniramineMaleate DextromethorphanHydrobromide is a combination medicine used in the treatment of dry cough. It relieves allergic symptoms such as sneezing  runny nose  watery eyes  and throat irritation. It also provides relief from congestion or stuffiness in the nose. It is taken with or without food in a dose and duration as advised by the doctor. The dose you are given will depend on your condition and how you respond to the medicine. You should keep taking this medicine for as long as your doctor recommends. If you stop treatment too early your symptoms may come back and your condition may worsen. Let your healthcare team know about all other medications you are taking as some may affect  or be affected by this medicine. The most common side effects are nausea  vomiting  loss of appetite  and headache. Most of these are temporary and usually resolve with time. Contact your doctor straight away if you are at all concerned about any of these side effects. Thi

In [ ]:
%%time
# https://stackoverflow.com/questions/59688148/spacy-ner-train-a-model-only-having-a-collection-of-entities

# DEFINE THE EPOCH
epoch = 5
NEW_TRAIN_DATA = TRAIN_DATA[0:5000]

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

    # Training for iterationsa
    for iteration in range(epoch):
        # shuufling examples  before every iteration
        random.shuffle(NEW_TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        # batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        # batches = spacy.util.minibatch(NEW_TRAIN_DATA, size=2)
        # print(batches)
        # # for batch in tqdm(batches):
        # for raw_text, entity_offsets in NEW_TRAIN_DATA:
        #     # texts, annotations = zip(*batch)
        #     # create Example
        #     doc = nlp.make_doc(raw_text)
        #     example = Example.from_dict(doc, entity_offsets)
        #     # Update the model
        #     nlp.update([example], losses=losses, drop=0.45)
        #     # print("Losses", losses)
        # batch up the examples using spaCy's minibatch
        # batches = minibatch(NEW_TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        # for batch in batches:
        #     texts, annotations = zip(*batch)
        #     nlp.update(
        #                 texts,  # batch of texts
        #                 annotations,  # batch of annotations
        #                 drop=0.5,  # dropout - make it harder to memorise data
        #                 losses=losses,
        #             )
        #     print("Losses", losses)
        for batch in spacy.util.minibatch(NEW_TRAIN_DATA, size=2):
            for text, annotations in batch:
                # print(annotations)
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # print(spacy.training.offsets_to_biluo_tags(doc, annotations['entities']))
                # Update the model
                nlp.update([example], losses=losses, drop=0.45)
                print("Losses", losses)


Losses {'ner': 5.917769303798105e-33}
Losses {'ner': 5.9198335279471834e-33}
Losses {'ner': 1.23704960812792e-26}
Losses {'ner': 1.2370496082412792e-26}
Losses {'ner': 1.2370496144728091e-26}
Losses {'ner': 1.2578140064269899e-26}
Losses {'ner': 2.210724196132325e-20}
Losses {'ner': 2.2107243634818285e-20}
Losses {'ner': 1.8900620647912034e-19}
Losses {'ner': 1.890062064791204e-19}
Losses {'ner': 1.8900620647942969e-19}
Losses {'ner': 1.8900620647954308e-19}
Losses {'ner': 2.1498768101911382e-16}
Losses {'ner': 2.1498768101911382e-16}
Losses {'ner': 2.1498768101914165e-16}
Losses {'ner': 2.1498768101914175e-16}
Losses {'ner': 2.1498768259712938e-16}
Losses {'ner': 2.1498768259712938e-16}
Losses {'ner': 2.1498768260463714e-16}
Losses {'ner': 2.1498768452158393e-16}
Losses {'ner': 2.1498768452158393e-16}
Losses {'ner': 2.1498768452158393e-16}
Losses {'ner': 2.1498768452262538e-16}
Losses {'ner': 2.1498768452262538e-16}
Losses {'ner': 2.149876845226423e-16}
Losses {'ner': 2.14987684769138

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Bambuterol is used to relieve symptoms of asthma a..." with entities "[(0, 10, 'Drug_Salt_Form'), (262, 272, 'Drug_Salt_...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner': 4.000004691721626}
Losses {'ner':

In [ ]:
# #nlp1  =spacy.load('/content/gdrive/MyDrive/Models/spacy_cust_model')
# nlp = spacy.load('/content/gdrive/MyDrive/Models/spacy_cust_model')

In [ ]:
 sen = """Medicines: 
1. Ascoril D Plus Syrup Sugar Free : 
4 ml 3 times a day x3 days (O--O--O) 
Fexofenadine 
2. Syrup Clavidur DDS :  Chlorpheniramine 
2.5 ml 2 times a day x 5 days (O--O) 
Amitriptyline+Chlordiazepoxide	Amoxycillin Clavulanic Acid
Amoxycillin
Clavulanic Acid
3. Cream Silverex : 
2 times a day x 5 days (O--O) 
4. Syrup Zyrtec : Fexofenadine	
4 ml 2 times a day x 3 days (O--O)"""

In [ ]:
doc = nlp(sen)
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# Save the model
output_dir = "/content/gdrive/MyDrive/Models/drug_salt_model"
if output_dir is not None:
   output_dir = Path(output_dir)
   if not output_dir.exists():
       output_dir.mkdir()
nlp.to_disk(output_dir)
print("Saved model to", output_dir)